### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Read School and Student Data File and store into Pandas Data Frames
schools = pd.read_csv("data/schools_complete.csv")
students = pd.read_csv("data/students_complete.csv")

# Combine the data into a single dataset
data = pd.merge(students, schools, how="left", on="school_name")
data.head(2)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# When you perform unique on Pandas Series it turns to Numpy array.
# This is the reason we are able to use np.ndarray.size attribute.
# total_schools = data['school_name'].unique().size
# total_students = data['Student ID'].unique().size
total_schools = data['school_name'].count()
total_students = data['Student ID'].count()
total_budget =  data['budget'].unique().sum()

# Find average math and reading scores.
average_math_score = data['math_score'].mean()
average_reading_score = data['reading_score'].mean()

# Use np.sum() to only count True elements in boolean array.
passing_math = np.sum(data['math_score'] >= 70) / total_students
passing_reading = np.sum(data['reading_score'] >= 70) / total_students
overall_average_rate = (average_math_score + average_reading_score) / 2

# Need to manually put index if all values are scalars
# and none are arrays.
district_summary = pd.DataFrame({
    'Total Schools': total_schools,
    'Total Students': f'{total_students:,}',
    'Total Budget': f'${total_budget:,.2f}',
    'Average Math Score': f'{average_math_score:.6f}',
    'Average Reading Score': f'{average_reading_score:.6f}',
    '% Passing Math': f'{100 * passing_math:.6f}',
    '% Passing Reading': f'{100 * passing_reading:.6f}',
    '% Overall Average Score': f'{overall_average_rate:.6}',
}, index=[0])
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Average Score
0,39170,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,80.4316


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:
# Create group by school name and use keys() to get list of school names
group = data.groupby(['school_name'])
school_name = group.groups.keys()

# Use explode() function to unflatten list of lists.
# Use size() function to get length of each group.
school_type = group['type'].unique().explode()
total_students = group.size()
total_budget = group['budget'].unique().explode()

# Note: Not allowed to divide list array by pd.Series
budget_per_student = total_budget / total_students
average_math = group['math_score'].mean()
average_reading = group['reading_score'].mean()

# Take each group array, set condition, and sum up all True values in array.
# Also, underscore is used because key in group is not being used.
passing_math = np.array([np.sum(val >= 70) for (_, val) in group['math_score']])
passing_reading = np.array([np.sum(val >= 70) for (_, val) in group['reading_score']])
overall_passing = (passing_math + passing_reading) / 2

school_summary = pd.DataFrame({
    'School Type':  school_type,
    'Total Students': total_students,
    'Total Budget': [f'${x:,.2f}' for x in total_budget],
    'Per Student Budget': [f'${x:,.2f}' for x in budget_per_student],
    'Average Math Score': average_math,
    'Average Reading Score': average_reading,
    '% Passing Math': 100 * passing_math / total_students,
    '% Passing Reading': 100 * passing_reading / total_students,
    '% Overall Passing Rate': 100 * overall_passing / total_students
}, index=school_name)
school_summary.sort_values(by='% Overall Passing Rate', ascending=False).head(5)

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
school_summary.sort_values(by='% Overall Passing Rate', ascending=True).head(5)

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
# Create group by school name and grade.
group = data.groupby(['school_name', 'grade'])

# This creates a pd.Series with two indices (MultiIndex).
math_scores = group['math_score'].mean()

# Remove index names: school_name and grade.
math_scores.index.names = [None] * 2

# Unstack MultiIndex dataframe and rearrange columns
# by numeric, grade order.
math_scores.unstack()[['9th', '10th', '11th', '12th']]

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
# This creates a pd.Series with two indices (MultiIndex).
reading_scores = group['reading_score'].mean()

# Remove index names: school_name and grade.
reading_scores.index.names = [None] * 2

# Unstack MultiIndex dataframe and rearrange columns
# by numeric, grade order.
reading_scores.unstack()[['9th', '10th', '11th', '12th']]

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# Remove unnecessary repeated code lines. DRY coding method.
def scores_by(dataframe, colname):
    # Group by spending ranges and take averages of precalculated values
    group = dataframe.groupby(colname)
    average_math = group['Average Math Score'].mean()
    average_reading = group['Average Reading Score'].mean()
    passing_math = group['% Passing Math'].mean()
    passing_reading = group['% Passing Reading'].mean()
    overall_passing = group['% Overall Passing Rate'].mean()

    # No need to set index. It is the same for all columns.
    return pd.DataFrame({
        'Average Math Score': average_math,
        'Average Reading Score': average_reading,
        '% Passing Math': passing_math,
        '% Passing Reading': passing_reading,
        '% Overall Passing Rate': overall_passing
    })

In [8]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Make deep copy to reset dataframe. Otherwise, we get error
# trying to take replace() attribute of float in next line
# when running same cell multiple times.
df = school_summary.copy(deep=True)

# Remove dollar sign we place in first part of District Summary
df['Per Student Budget'] = [float(x.replace('$', '')) for x in df['Per Student Budget']]

# Assign label bin to each row in dataframe.
df['Spending Ranges (Per Student)'] = pd.cut(
    df['Per Student Budget'],
    spending_bins,
    labels=group_names
)

# Use custom scores_by function to print out desired table.
scores_by(df, 'Spending Ranges (Per Student)')

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Assign label bin to each row in dataframe.
school_summary['School Size'] = pd.cut(
    school_summary['Total Students'],
    size_bins,
    labels=group_names
)

# Use custom scores_by function to print out desired table.
scores_by(school_summary, 'School Size')

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [10]:
# Use custom scores_by function to print out desired table.
scores_by(school_summary, 'School Type')

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
